# Appendix 10.2: Tool Use

（付録 10.2：ツール利用（Tool Use / Function Calling））

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## Setup

次のセットアップセルを実行して、APIキーの読み込みと `get_completion` ヘルパー関数の準備を行います。

In [5]:
# %pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

# Override the MODEL_NAME variable in the IPython store to use Sonnet instead of the Haiku model
MODEL_NAME='anthropic.claude-3-sonnet-20240229-v1:0'
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

# Rewrittten to call Claude 3 Sonnet, which is generally better at tool use, and include stop_sequences
# def get_completion(messages, system_prompt="", prefill="",stop_sequences=None):
#     message = client.messages.create(
#         model=MODEL_NAME,
#         max_tokens=2000,
#         temperature=0.0,
#         messages=messages,
#         system=system_prompt,
#         stop_sequences=stop_sequences
#     )
#     return message.content[0].text

def get_completion(messages, system_prompt="", prefill="",stop_sequences=None):
    message = ''
    stream = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages,
        system=system_prompt,
        stop_sequences=stop_sequences,
        stream=True
    )
    for event in stream:
        if event.type == "content_block_delta":
            print(event.delta.text, flush=True, end="")
            message += event.delta.text
    return message

---

## Lesson

最初は概念的に複雑に見えるかもしれませんが、ツール利用（tool use）、別名 function calling は実はとてもシンプルです！ ツール利用を実装するのに必要なスキルは、すでにこのチュートリアルで学んでいます。要するに **置換（substitution）＋プロンプトチェイニング** の組み合わせです。

これまでの置換演習では、テキストをプロンプトに差し込みました。ツール利用では、ツール／関数の実行結果をプロンプトに差し込みます。Claude 自身がツールや関数を実際に呼び出したりアクセスしたりできるわけではありません。代わりに次の流れで行います：
1. Claude に「呼びたいツール名」と「引数」を出力させる
2. ツールを呼び出している間は、それ以上の応答生成を止める
3. ツール結果を会話に追記し、それを含めてもう一度 Claude に促す（reprompt）

function calling が便利なのは、Claude の能力を拡張し、より複雑で多段階のタスクを扱えるようにできるからです。
Claude に与えられる関数（ツール）の例：
- 電卓（Calculator）
- 単語数カウンター（Word counter）
- SQL データベースのクエリ実行・データ取得
- 天気 API

Claude にツール利用をさせるには、次の2要素を組み合わせます：

1. **システムプロンプト**：ツール利用の概念を説明し、この状況でアクセス可能なツールの詳細な説明リストを Claude に与える
2. **制御ロジック**：Claude のツール利用要求をオーケストレーションし、実際に実行するためのロジック

### Tool use roadmap

*このレッスンでは現時点のツール利用フォーマットを扱います。ただし、近い将来ツール利用機能はアップデート／改善される予定で、たとえば次が含まれます：*
* *関数定義・呼び出しのより簡潔なフォーマット*
* *より堅牢なエラーハンドリングとエッジケース対応*
* *API 全体とのより強い統合*
* *特に複雑なツール利用タスクにおける信頼性・性能の向上*

### Examples

Claude でツール利用を有効にするには、まずシステムプロンプトから始めます。この「ツール利用用システムプロンプト」では、Claude に次を伝えます：
* ツール利用の基本的な前提と、何をするものか
* Claude が与えられたツールをどう呼び出し、どう使うか
* この状況でアクセスできるツールの詳細なリスト

以下は、Claude にツール利用を説明するシステムプロンプトの前半部分です。この部分は、ツール利用をさせるあらゆる場面で汎用的に使えます。ここで Claude に与えるツール呼び出し構造（`<function_calls> [...] </function_calls>`）は、Claude がこの形式で出力するよう特別に学習されているため、基本的にはこの形に従うことをおすすめします。

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

In [2]:
system_prompt_tools_general_explanation = """あなたは、ユーザーの質問に答えるために使用できる一連の関数にアクセスできます。これには、
サンドボックス化された計算環境へのアクセスも含まれます。ただし、以下の関数を呼び出す場合を除き、
現時点ではファイルを検査したり、外部リソースとやり取りしたりすることはできません。

ユーザーへの返信の一部として、次のような "<function_calls>" ブロックを書くことで、
1つ以上の関数を呼び出すことができます：
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<invoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

文字列およびスカラー型のパラメータはそのまま指定し、リストやオブジェクトは JSON 形式を使用してください。
なお、文字列値に含まれるスペースは削除されません。出力は有効な XML である必要はなく、
正規表現によって解析されます。

出力および／またはエラーは、その後の "<function_results>" ブロックに表示され、
ユーザーへの返信の一部としてそこに保持されます。
その後、ユーザーへの返信の残りを続けて記述したり、エラーに対応したり、
必要に応じて追加の関数呼び出しを行ったりすることができます。
もし関数呼び出しの後に "<function_results>" が表示されない場合は、
それらが不正な形式で記述されており、関数呼び出しとして認識されていない可能性があります。"""


次はシステムプロンプトの後半で、この状況で Claude がアクセスできる具体的なツール定義を記述します。この例では、3つのパラメータ（2つのオペランドと演算子）を取る電卓ツール（calculator）を Claude に与えます。

最後に、システムプロンプトの前半と後半を結合します。

In [3]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

これで、`calculator` ツールの利用が必要な質問を Claude に与えられます。Claude が関数を呼び出したかどうか（いつ呼び出したか）を検出するため、`stop_sequences` に `<function_calls\>` を使います。

In [6]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
# print(function_calling_response)

Okay, let's multiply those two numbers:

<function_calls>
<invoke name="calculator">
<parameter name="first_operand">1984135</parameter>
<parameter name="second_operand">9343116</parameter>
<parameter name="operator">*</parameter>
</invoke>


次に、Claude の function call からパラメータを取り出し、Claude の代わりに実際に関数（ツール）を実行します。

まずは関数本体（コード）を定義します。

In [7]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

次に、Claude の function call 応答からパラメータを抽出します。必要なパラメータが揃っていれば、calculator ツールを実行します。

In [8]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

---------------- RESULT ----------------
18,538,003,464,660


結果が得られたら、その結果を適切にフォーマットして Claude に渡す必要があります。そうすることで、Claude はこの結果が「どのツールに対する結果なのか」を理解できます。このための定型フォーマットがあり、Claude はこの形式を認識するよう学習されています：
```
<function_results>
<result>
<tool_name>{TOOL_NAME}</tool_name>
<stdout>
{TOOL_RESULT}
</stdout>
</result>
</function_results>
```

下のセルを実行して、ツール結果をこの構造に整形しましょう。

In [10]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

<function_results>
<result>
<tool_name>do_pairwise_arithmetic</tool_name>
<stdout>
18538003464660
</stdout>
</result>
</function_results>


あとは、この結果を先ほどと同じメッセージチェーンに追記して Claude に返すだけです。これで一連の流れが完成です！

In [11]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Claude's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
# print(final_response)

The result of multiplying 1,984,135 by 9,343,116 is 18,538,003,464,660.

To break this down:
1,984,135
x 9,343,116
-----------
17,857,215,000 (1,984,135 x 9,000,000)
   679,280,460 (1,984,135 x 343,116)
--------------
18,538,003,464,660

So the calculator function correctly multiplied the two large numbers. Let me know if you need any other calculations!------------- FINAL RESULT -------------


ツール利用チェーンを最初から最後まで一通り動かせました。おめでとうございます！

では次に、与えたツールをまったく使う必要がない質問を Claude に与えたらどうなるでしょう？

In [12]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
# print(function_calling_response)

The capital of France is Paris.

成功です！ 見てのとおり、Claude は不要なときには関数を呼び出さない判断ができました。

上の内容を変えずにレッスンのプロンプトだけ試したい場合は、ノートブック末尾の [**Example Playground**](#example-playground) を利用してください。

---

## Exercises
- [Exercise 10.2.1 - SQL](#exercise-1021---SQL)

### Exercise 10.2.1 - SQL
この演習では、世界最小（？）の「データベース」に対して、クエリ実行と書き込みを行うためのツール利用プロンプトを作ります。ここでいうデータベースは、実際には単なる辞書（dictionary）です。

In [13]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

次に、データベースへの書き込み／読み出しを行う関数のコードです。

In [14]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

演習を解くには、上の `system_prompt_tools_specific_tools` のようなシステムプロンプト定義から始めてください。各ツールについて「名前」と「説明」を含め、さらに各関数の各パラメータについて「名前」「型」「説明」を必ず書きます。下に開始用の足場（scaffolding）を用意しています。

In [16]:
system_prompt_tools_specific_tools_sql = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>get_user</tool_name>
<description>
Get a user from the database.
</description>
<parameters>
<parameter>
<name>user_id</name>
<type>int</type>
<description>The ID of the user to get.</description>
</parameter>
</parameters>
</tool_description>

<tool_description>
<tool_name>get_product</tool_name>
<description>
Get a product from the database.
</description>
<parameters>
<parameter>
<name>product_id</name>
<type>int</type>
<description>The ID of the product to get.</description>
</parameter>
</parameters>
</tool_description>

<tool_description>
<tool_name>add_user</tool_name>
<description>
Add a user to the database.
</description>
<parameters>
<parameter>
<name>name</name>
<type>str</type>
<description>The name of the user to add.</description>
</parameter>
<parameter>
<name>email</name>
<type>str</type>
<description>The email of the user to add.</description>
</parameter>
</parameters>
</tool_description>

<tool_description>
<tool_name>add_product</tool_name>
<description>
Add a product to the database.
</description>
<parameters>
<parameter>
<name>name</name>
<type>str</type>
<description>The name of the product to add.</description>
</parameter>
<parameter>
<name>price</name>
<type>float</type>
<description>The price of the product to add.</description>
</parameter>
</parameters>
</tool_description>

</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

準備ができたら、下の例でツール定義システムプロンプトを試してみましょう。次のセルを実行するだけです！

In [18]:
examples = [
    "Add a user to the database named Deborah.",
    "Add a product to the database named Thingo",
    "Tell me the name of User 2",
    "Tell me the name of Product 3"
]

for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # Get & print Claude's response
    function_calling_response = get_completion([message], system_prompt=system_prompt, stop_sequences=stop_sequences)
    # print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

Okay, let's add a user named Deborah to the database:

<function_calls>
<invoke name="add_user">
<parameter name="name">Deborah</parameter>
<parameter name="email">deborah@example.com</parameter>
</invoke>
Okay, let's add a new product to the database:

<function_calls>
<invoke name="add_product">
<parameter name="name">Thingo</parameter>
<parameter name="price">10.0</parameter>
</invoke>
To get the name of User 2, we can use the `get_user` function like this:

<function_calls>
<invoke name="get_user">
<parameter name="user_id">2</parameter>
</invoke>
Okay, let me get the product with ID 3 from the database:

<function_calls>
<invoke name="get_product">
<parameter name="product_id">3</parameter>
</invoke>


うまくできていれば、function calling のメッセージが `add_user`、`add_product`、`get_user`、`get_product` 関数を正しく呼び出すはずです。

追加課題として、コードセルを追加してパラメータ解析（parameter-parsing）のコードを書いてみてください。その上で、Claude が提示したパラメータを使って関数を実際に実行し、呼び出し後の「データベース」の状態を確認してみましょう。

❓ 参考となる解答例を見たい場合は、次のセルを実行してください！

In [15]:
print(hints.exercise_10_2_1_solution)

system_prompt = system_prompt_tools_general_explanation + """Here are the functions available in JSONSchema format:

<tools>

<tool_description>
<tool_name>get_user</tool_name>
<description>
Retrieves a user from the database by their user ID.
</description>
<parameters>
<parameter>
<name>user_id</name>
<type>int</type>
<description>The ID of the user to retrieve.</description>
</parameter>
</parameters>
</tool_description>

<tool_description>
<tool_name>get_product</tool_name>
<description>
Retrieves a product from the database by its product ID.
</description>
<parameters>
<parameter>
<name>product_id</name>
<type>int</type>
<description>The ID of the product to retrieve.</description>
</parameter>
</parameters>
</tool_description>

<tool_description>
<tool_name>add_user</tool_name>
<description>
Adds a new user to the database.
</description>
<parameters>
<parameter>
<name>name</name>
<type>str</type>
<description>The name of the user.</description>
</parameter>
<parameter>
<name>em

### Congrats!

ツール利用（tool use）と function calling の学習、おめでとうございます！ Search（検索）と RAG についてさらに学びたい場合は、最後の付録セクションに進んでください。

---

## Example Playground

このエリアでは、このレッスンで示したプロンプト例を自由に試し、プロンプトを調整して Claude の応答がどう変わるかを観察できます。

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Claude's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)